In [ ]:
#Working on local machine (i.e. not colab or kaggle)

In [ ]:
!pip install torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0
!pip install fastai==2.6.3 --no-deps
!pip install self_supervised

In [ ]:
import fastai
import self_supervised
import torch
print(self_supervised.__version__)
fastai.__version__ 

1.0.4


'2.7.9'

In [ ]:
torch.__version__

In [ ]:
from fastai.vision.all import *
from self_supervised.augmentations import *
from self_supervised.layers import *
import inspect
import warnings
import random
import math
warnings.filterwarnings("ignore")
#from Base_Stein.SVGD_classes import *

In [ ]:
#Like most other SSL algorithms BT's model consists of an encoder and projector (MLP) layer.
#Definition is straightforward:
#https://colab.research.google.com/github/KeremTurgutlu/self_supervised/blob/master/nbs/14%20-%20barlow_twins.ipynb#scrollTo=1M6QcUChcvpz
class BarlowTwinsModel(Module):
    """An encoder followed by a projector
    """
    def __init__(self,encoder,projector):self.encoder,self.projector = encoder,projector
        
    def forward(self,x): return self.projector(self.encoder(x))

In [ ]:
#HOWEVER instead of directly using the above, by passing both an encoder and a projector, create_barlow_twins_model
#function can be used by minimally passing a predefined encoder and the expected input channels.

#In the paper it's mentioned that MLP layer consists of 3 layers... following function will create a 3 layer
#MLP projector with batchnorm and ReLU by default. Alternatively, you can change bn and nlayers. 

#Questions: Why torch.no_grad() when doing this?
def create_barlow_twins_model(encoder, hidden_size=256, projection_size=128, bn=True, nlayers=3):
    "Create Barlow Twins model"
    n_in  = in_channels(encoder)
    with torch.no_grad(): representation = encoder(torch.randn((2,n_in,128,128)))
    projector = create_mlp_module(representation.size(1), hidden_size, projection_size, bn=bn, nlayers=nlayers) 
    apply_init(projector)
    return BarlowTwinsModel(encoder, projector)

#Similar to above. Simple API to make the BT model:

In [ ]:
#BarlowTwins Callback
#The following parameters can be passed:
# - aug_pipelines
# Imb lambda is the weight for redundancy reduction term in the loss function

@delegates(get_multi_aug_pipelines)
def get_barlow_twins_aug_pipelines(size,**kwargs): return get_multi_aug_pipelines(n=2,size=size,**kwargs)

In [ ]:
#Uniform random number between a and b
def Unif(a,b):
    return (b-a)*torch.rand(1).item()+a

In [ ]:
def random_polynomial(A):
    
    #B=torch.normal(mean=0, std=0.025, size=(1, 1)).item() #First Horner term (and is coefficient of x^4)
    
    #Btem = torch.normal(mean=0,std=0.05, size=(1, 1)).item() #Sample coefficient of x^3
    #B = Btem + B*A #Third Horner term
    
    B = torch.normal(mean=0,std=0.5, size=(1, 1)).item() #Sample coefficient of x^2
    
#     Btem = torch.normal(mean=0,std=0.5, size=(1, 1)).item() #Sample coefficient of x^2
#     B = Btem + B*A #Third Horner term
    
    Btem = random.choice([-1,1])*torch.normal(mean=0,std=1, size=(1, 1)).item() #Sample coefficient of x
    #Btem=1
    B = Btem + B*A #Fourth Horner term
    
    Btem = torch.normal(mean=0,std=1, size=(1, 1)).item() #Sample coefficient of x^0
    B = Btem + B*A #Fifth Horner term
    
    
    return B

In [ ]:
def random_polynomial_bestsofar(A):
    
    
    B=torch.normal(mean=0, std=0.025, size=(1, 1)).item() #First Horner term (and is coefficient of x^4)
    
    Btem = torch.normal(mean=0,std=0.05, size=(1, 1)).item() #Sample coefficient of x^3
    B = Btem + B*A #Third Horner term
    
    Btem = torch.normal(mean=0,std=0.5, size=(1, 1)).item() #Sample coefficient of x^2
    B = Btem + B*A #Third Horner term
    
    Btem = random.choice([-1,1])*torch.normal(mean=1,std=2, size=(1, 1)).item() #Sample coefficient of x
    B = Btem + B*A #Fourth Horner term
    
    Btem = torch.normal(mean=0,std=1, size=(1, 1)).item() #Sample coefficient of x^0
    B = Btem + B*A #Fifth Horner term
    
    
    return B

In [ ]:
def low_deg(A):
    power=Unif(1,1.25)
    coeff2 = torch.normal(mean=0, std=1, size=(1, 1)).item() #degree 2 term
    coeff1 = torch.normal(mean=1, std=1, size=(1, 1)).item() #degree 2 term
    
#     coeff2 = torch.normal(mean=0, std=1, size=(1, 1)).item() #degree 2 term
#     coeff1 = torch.normal(mean=1, std=0.7, size=(1, 1)).item() #degree 2 term

#     coeff2 = torch.normal(mean=0, std=1, size=(1, 1)).item() #degree 2 term
#     coeff1 = torch.normal(mean=0, std=1, size=(1, 1)).item() #degree 2 term

    
    B = (coeff1*A + coeff2*torch.abs(A).pow(power))
    
    #B = (1/power)*torch.abs(A).pow(power)
    return B

In [ ]:
def random_quintic(A):

    
    B=torch.normal(mean=0, std=0.125, size=(1, 1)).item() #First Horner term (and is coefficient of x^4)
    
    Btem=torch.normal(mean=0, std=0.25, size=(1, 1)).item()#Sample coefficient of x^3
    B = Btem + B*A #Second Horner term
    
    Btem = torch.normal(mean=0,std=0.5, size=(1, 1)).item() #Sample coefficient of x^2
    B = Btem + B*A #Third Horner term
    
    Btem = random.choice([-1,1])*torch.normal(mean=1,std=2, size=(1, 1)).item() #Sample coefficient of x
    B = Btem + B*A #Fourth Horner term
    
    Btem = torch.normal(mean=0,std=1, size=(1, 1)).item() #Sample coefficient of x^0
    B = Btem + B*A #Fifth Horner term
    
    
    return B

In [ ]:
def random_sinusoid(x,std=2.0):
    
    t=torch.normal(mean=1,std=std,size=(1,1)).item()
    s=torch.normal(mean=1,std=std,size=(1,1)).item()
    
    u=torch.normal(mean=0,std=std,size=(1,1)).item()
    v=torch.normal(mean=0,std=std,size=(1,1)).item()
    
    a=torch.normal(mean=0,std=2,size=(1,1)).item()
    b=torch.normal(mean=0,std=2,size=(1,1)).item()
    
    return a*torch.sin(t*x+s) + b*torch.cos(u*x + v)

In [ ]:
def poly_sinusoid(x):
    
    return (x) + 0.2*random_sinusoid(x,std=(0.5))

In [ ]:
#export
class BarlowTwins(Callback):
    order,run_valid = 9,True
    def __init__(self, aug_pipelines, lmb=5e-3, print_augs=False):
        assert_aug_pipelines(aug_pipelines)
        self.aug1, self.aug2 = aug_pipelines
        if print_augs: print(self.aug1), print(self.aug2)
        store_attr('lmb')
        
    def before_fit(self): 
        self.learn.loss_func = self.lf
        nf = self.learn.model.projector[-1].out_features
        self.I = torch.eye(nf).to(self.dls.device)
            
    def before_batch(self):
        xi,xj = self.aug1(self.x), self.aug2(self.x)
        self.learn.xb = (torch.cat([xi, xj]),)
        
        #Uncomment to run standard BT
    
#     def lf(self, pred, *yb): #pred is (bs+bs)*projection_size
#         bs,nf = pred.size(0)//2,pred.size(1)

#         z1, z2 = pred[:bs],pred[bs:] #so z1 is bs*projection_size, likewise for z2

#         z1norm = (z1 - z1.mean(0)) / z1.std(0, unbiased=False)
#         z2norm = (z2 - z2.mean(0)) / z2.std(0, unbiased=False)
        
#         C = (z1norm.T @ z2norm) / bs 
#         cdiff = (C - self.I)**2
#         loss = (cdiff*self.I + cdiff*(1-self.I)*self.lmb).sum() 
#         return loss


    def lf(self, pred, *yb): #pred is (bs+bs)*projection_size
        bs,nf = pred.size(0)//2,pred.size(1)

        #All standard, from BT
        z1, z2 = pred[:bs],pred[bs:] #so z1 is bs*projection_size, likewise for z2
        z1norm = (z1 - z1.mean(0)) / z1.std(0, unbiased=False)
        z2norm = (z2 - z2.mean(0)) / z2.std(0, unbiased=False)
        
        
        C = (z1norm.T @ z2norm) / bs 
        cdiff = (C - self.I)**2
    
        #polyprob=0.1
        polyprob=0.5
        temrand = random.random()
        if temrand < polyprob: #With some probability we want off diag terms to be (quadratic) say.

            p=Unif(1,2.5) 
            z1norm_2 = (1/p)*torch.abs(z1norm).pow(p)
            z2norm_2 = z2norm
                
            C_2 = (z1norm_2.T @ z2norm_2) / bs
            
            cdiff_2 = (C_2)**2 #don't need to subtract I as only looking at off diag terms
            
        else:
            cdiff_2 = cdiff
            
        l2 = cdiff_2*(1-self.I)*self.lmb #Is either the standard term - or not.

        loss = (cdiff*self.I + l2).sum() 
        return loss

    
    @torch.no_grad()
    def show(self, n=1):
        bs = self.learn.x.size(0)//2
        x1,x2  = self.learn.x[:bs], self.learn.x[bs:] 
        idxs = np.random.choice(range(bs),n,False)
        x1 = self.aug1.decode(x1[idxs].to('cpu').clone()).clamp(0,1)
        x2 = self.aug2.decode(x2[idxs].to('cpu').clone()).clamp(0,1)
        images = []
        for i in range(n): images += [x1[i],x2[i]] 
        return show_batch(x1[0], None, images, max_n=len(images), nrows=n)

In [ ]:
#Debugging cell - delete later (similar to cell below)
ps=500
hs=500
fastai_encoder = create_encoder('xresnet18', n_in=1, pretrained=False)
model = create_barlow_twins_model(fastai_encoder, hidden_size=hs,projection_size=ps)# projection_size=1024)
#So aside from size, randomresizedcrop takes in two args: resize_scale and resize_ratio. So we want to put in 
#values for these which is tantamount to doing nothing
#So if we choose resize_scale=(1,1) then the images look the same.
#IMPORTANT: So this aug pipelines, insofar as I can tell at the moment, is tantamount to "do nothing"
aug_pipelines = get_barlow_twins_aug_pipelines(size=28, rotate=True,flip_p=0,resize_scale=(0.7,1), jitter=False, bw=False,blur=True,blur_p=0.5,blur_s=8, stats=None, cuda=True)
#learn = Learner(dls, model,ShortEpochCallback(0.001), cbs=[BarlowTwins(aug_pipelines, print_augs=True)])
learn = Learner(dls, model, cbs=[BarlowTwins(aug_pipelines, print_augs=True)])
learn.fit(100) #300                                        


KeyboardInterrupt



In [ ]:
# #Get the dataloader and set batch size 
# ts=512 #training set size
# bs=256 
# device='cpu'
# path = untar_data(URLs.MNIST)
# items = get_image_files(path/'training') #i.e. NOT testing!!!
# items=items.shuffle()

# items1 = items[0:ts]
# split = RandomSplitter(valid_pct=0.5) #randomly split training set into training and validation
# #tds = Datasets(items,splits=split(items)) #Do we want this?
# tds = Datasets(items1, [PILImageBW.create, [parent_label, Categorize()]], splits=split(items1)) #Or do we want this?
# dls = tds.dataloaders(bs=bs, after_item=[ToTensor(), IntToFloatTensor()], device=device)

# #Evaluate linear classifier on this guy
# items2 = items[ts:]
# split = RandomSplitter(valid_pct=0.99) #randomly split training set into training and validation
# tds_new = Datasets(items2, [PILImageBW.create, [parent_label, Categorize()]], splits=split(items2)) #Or do we want this?
# dls_new = tds_new.dataloaders(bs=bs, after_item=[ToTensor(), IntToFloatTensor()], device=device)

In [ ]:
def seed_everything(seed=42):
    """"
    Seed everything.
    """   
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

def tune_set(items0,tune_s=1000):
  
    items0=items0.shuffle()
    d = {'0':0,'1':0,'2':0,'3':0,'4':0,'5':0,'6':0,'7':0,'8':0,'9':0}
    ITEMS=[]
    for i in items0:
        s=str(i).split('/training/')[1][0]
        if d[s] is 0 or d[s] is 1:
            ITEMS.append(i)
            d[s]+=1
    #items0=ITEMS

    for i in items0:
        if i not in ITEMS:
            ITEMS.append(i)
            
    split = IndexSplitter(list(range(20)))

    tds_tune = Datasets(ITEMS, [PILImageBW.create, [parent_label, Categorize()]], splits=split(ITEMS)) #Or do we want this?
    dls_tune = tds_tune.dataloaders(bs=20, after_item=[ToTensor(), IntToFloatTensor()], device=device)
    
    return dls_tune

 

In [ ]:
#Get the dataloader and set batch size 
ts=16384 #training set size - most everything
bs=512
device='cpu'
path = untar_data(URLs.MNIST)

items = get_image_files(path/'training') #i.e. NOT testing!!!
items.sort() 

seed_everything(seed=42)
# import random
# random.seed(20)
items=items.shuffle()

items1 = items[0:ts] #train on these guys
split = RandomSplitter(valid_pct=0.0) #randomly split training set into training and validation
#tds = Datasets(items,splits=split(items)) #Do we want this?
tds = Datasets(items1, [PILImageBW.create, [parent_label, Categorize()]], splits=split(items1)) #Or do we want this?
dls = tds.dataloaders(bs=bs, after_item=[ToTensor(), IntToFloatTensor()], device=device)

tune_s=1000 #we choose 20 guys (randomly) out of 1000 to tune on
items0 = items[ts:ts+tune_s] #for fine tuning - just choose 2000 guys to extract 20 for fine tuning 
dls_tune=tune_set(items0,tune_s=tune_s)

#NB: Uncomment and compare in colab and kaggle
for x,y in dls_tune.train:
    print(x.mean())
    input()
    break

dls_tune=tune_set(items0,tune_s=1000)

for x,y in dls_tune.train:
    print(x.mean())
    input()
    break


#Evaluate linear classifier on this guy
items2 = items[ts+tune_s:]
split = RandomSplitter(valid_pct=0.0) #randomly split training set into training and validation
tds_test = Datasets(items2, [PILImageBW.create, [parent_label, Categorize()]], splits=split(items2)) #Or do we want this?
dls_test = tds_test.dataloaders(bs=390, after_item=[ToTensor(), IntToFloatTensor()], device=device)


TensorImageBW(0.1257)

TensorImageBW(0.1301)


KeyboardInterrupt: Interrupted by user

In [ ]:
#Copy paste from GitHub - trying to debug error from using create_encoder(.) below

def _get_first_layer(m):
    "Access first layer of a model"
    c,p,n = m,None,None  # child, parent, name
    
    print(m)
    print(dir(m))
    
    input()
    m.named_parameters()
    input('ok')
    
    
    for n in next(m.named_parameters())[0].split('.')[:-1]:
        p,c=c,getattr(c,n)
    return c,p,n

def _update_first_layer(model, n_in, pretrained):
    "Change first layer based on number of input channels"
    if n_in == 3: return
    first_layer, parent, name = _get_first_layer(model)
    assert isinstance(first_layer, nn.Conv2d), f'Change of input channels only supported with Conv2d, found {first_layer.__class__.__name__}'
    assert getattr(first_layer, 'in_channels') == 3, f'Unexpected number of input channels, found {getattr(first_layer, "in_channels")} while expecting 3'
    params = {attr:getattr(first_layer, attr) for attr in 'out_channels kernel_size stride padding dilation groups padding_mode'.split()}
    params['bias'] = getattr(first_layer, 'bias') is not None
    params['in_channels'] = n_in
    new_layer = nn.Conv2d(**params)
    if pretrained:
        _load_pretrained_weights(new_layer, first_layer)
    setattr(parent, name, new_layer)
    
def create_body(model, n_in=3, pretrained=True, cut=None):
    "Cut off the body of a typically pretrained `arch` as determined by `cut`"
    _update_first_layer(model, n_in, pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    return cut_model(model, cut)

#export
def create_fastai_encoder(arch:str, pretrained=True, n_in=3, pool_type=PoolingType.CatAvgMax):
    "Create timm encoder from a given arch backbone"
    encoder = create_body(arch, n_in, pretrained, cut=None)
    pool = AdaptiveConcatPool2d() if pool_type == "catavgmax" else nn.AdaptiveAvgPool2d(1)
    return nn.Sequential(*encoder, pool, Flatten())

def create_timm_encoder(arch:str, pretrained=True, n_in=3, pool_type=PoolingType.CatAvgMax):
    "Creates a body from any model in the `timm` library. If pool_type is None then it uses timm default"
    if ('vit' in arch) or (pool_type is None):
        model = timm.create_model(arch, pretrained=pretrained, in_chans=n_in, num_classes=0)
    else:
        model = timm.create_model(arch, pretrained=pretrained, in_chans=n_in, num_classes=0, global_pool=pool_type)
    return model

def create_encoder(arch:str, pretrained=True, n_in=3, pool_type=PoolingType.CatAvgMax):
    "A utility for creating encoder without specifying the package"
    if arch in globals(): return create_fastai_encoder(globals()[arch], pretrained, n_in, pool_type)
    else:                 return create_timm_encoder(arch, pretrained, n_in, pool_type)

In [ ]:
fastai_encoder = create_encoder('xresnet18', n_in=1, pretrained=False)


<function xresnet18>
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


KeyboardInterrupt: Interrupted by user

In [ ]:
_get_first_layer??

In [ ]:
#A "reasonable" composite augmentation: initially copy pasted BT. We run this cell a few times to check it makes sense
#Also define encoder and model
#fastai_encoder = create_encoder('xresnet18', n_in=1, pretrained=False)
# model = create_barlow_twins_model(fastai_encoder, hidden_size=10,projection_size=10)# projection_size=1024)
# #So aside from size, randomresizedcrop takes in two args: resize_scale and resize_ratio. So we want to put in 
# #values for these which is tantamount to doing nothing
# #So if we choose resize_scale=(1,1) then the images look the same.
# #IMPORTANT: So this aug pipelines, insofar as I can tell at the moment, is tantamount to "do nothing"
# aug_pipelines = get_barlow_twins_aug_pipelines(size=28, rotate=True,flip_p=0,resize_scale=(0.7,1), jitter=False, bw=False,blur=True,blur_p=0.5,blur_s=8, stats=None, cuda=False)
# #learn = Learner(dls, model,ShortEpochCallback(0.001), cbs=[BarlowTwins(aug_pipelines, print_augs=True)])
# learn = Learner(dls, model, cbs=[BarlowTwins(aug_pipelines, print_augs=True)])

# #dls.valid.bs = len(dls.valid_ds) #Set the validation dataloader batch size to be the length of the validation dataset

# b = dls.one_batch()
# learn._split(b)
# learn('before_batch')
# axes = learn.barlow_twins.show(n=2)

In [ ]:
create_encoder??

In [ ]:
#Simple linear classifier
class LinearClassifier(nn.Module):
    
    def __init__(self,zdim):
        super().__init__()
        self.fc1 = nn.Linear(zdim,10) #As 10 classes for mnist
        
    def forward(self,x):
        x = cast(self.fc1(x),Tensor) #so we have to use cross entropy loss. cast is because using old version fastai 
        return x

In [ ]:

#Train Classifier on encoder(mnist) for (at the moment) one epoch

fastai_encoder.eval()

zdim=1024 #see above
head = LinearClassifier(zdim=zdim)
device='cuda'
head.to(device)
optimizer = torch.optim.Adam(head.parameters())
criterion = nn.CrossEntropyLoss()
#EPOCHS=100

for epoch in range(100):
    for x,y in dls_tune.valid:
        #break 
        #b = dls.train.one_batch() #Seems need dls[0] or dls.train for training ... dls[1] is validation see here https://docs.fast.ai/data.core.html#DataLoaders.__getitem__
        #x,y = b[0],b[1]

        loss = criterion(head(fastai_encoder(x)),y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #print(loss)
print('done')
        

In [ ]:

#Test result of above cell on the validation set - assumes that batch size of valid-dataloader is = number of valid samples                                        

# print('The validation batch size is: {} '.format(dls.valid.bs))
# input()

#b = dls.valid.one_batch()

fastai_encoder.eval()
N=len(dls_test.train)*bs #close to len(dls_test.train_ds) but not quite...

num_correct=0
for x,y in dls_test.train:

    ypred = head(fastai_encoder(x))
    correct = (torch.argmax(ypred,dim=1) == y).type(torch.FloatTensor)
    num_correct += correct.sum()
print(num_correct/N)
print('done')
print(num_correct)

random.seed(10)  
**NB: Baseline with BT:
random.seed(10), 500 epochs. 
BT: (0.7837,0.7964,0.7890)**  
Now, hacking experiments
100 learn epochs  
Quad_polyprob10 = (0.7265,0.7308)  
random_polynomial = (0.7233,0.7151)    
low_deg = (0.7153,0.7106).  
poly_sinusoid_polyprob20 = (0.7229,0.7250)  
random_quad_new2_polyprob100 = (0.6357) - this is not good.     
300 learn_epochs  
poly_sinusoid = 0.7541 
Can see above for other base hps

Comments Quad_polyprob10 is literally just .pow(2)

Note: We should at least try training linear classifier head on dls.valid (i.e. on different data to what BT was trained on - I think right?). When we do this we get:



Hacking since ran out of GPU
Base HPs: ts=512,bs=256, ps=hs=500,pp=1, learn_epochs=**200**, tune_epochs=**10**.
Run_1: BT=0.6795, MBT=0.7303

Base HPs as before: ts=512,bs=256, ps=hs=500,pp=1, learn_epochs=1000, tune_epochs=100.
See random_polynomial above for implementation

**Note: Using same random_polynomial as Version 6 notebook - so we record those 
(3) runs here and continue on given that mean was "best so far" (aside: although really best so far is ~ 0.92-ish, averaged over 5 or 6 runs (see some earlier notebooks). So we want to see how this random function does once we have 5 or 6 runs)**  
**Run_1: 0.9274, Run_2: 0.9309, Run_3: 0.9202, Run_4: 0.9302, Run_5: 0.9236**   
**Mean = 0.92646, which is basically what it was after 3 runs. Cool**

Comment: So ~92.5 is best mean performance so far. 92.6 with current system, got ~92.3-5 on some others. 



In [ ]:
from statistics import mean
mean([0.9274,0.9309,0.9202])#,0.9302,0.9236])

In [ ]:
# #Just train a linear classifier (no encoder)
# #Basically cell above but remove encoder and some re-shaping
zdim=500 #see above
head = LinearClassifier(zdim=zdim)
head.to(device)
optimizer = torch.optim.Adam(head.parameters())
criterion = nn.CrossEntropyLoss()


for x,y in dls.train:
    #break
    #b = dls.train.one_batch() #see here https://docs.fast.ai/data.core.html#DataLoaders.__getitem__
    #x,y = b[0],b[1]

    x=x.view(bs,zdim)
    x=cast(x, Tensor) #Have to do this when using old version of fastai for some reason...
    
    out = head(x)
    loss = criterion(out,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
# #Test result of above cell, (i.e. just a linear classifier), on the validation set - assumes that batch size of valid-dataloader is = number of valid samples                                        
# num_correct=0
# for x,y in dls_new.valid:

#     x=x.view(x.shape[0],zdim)
#     x=cast(x, Tensor) #Have to do this when using old version of fastai for some reason...
    
#     ypred = head(x)
#     correct = (torch.argmax(ypred,dim=1) == y).type(torch.FloatTensor)
#     num_correct += correct.sum()
    
# print(num_correct/len(dls_new.valid_ds))